#### Base de Datos (Database)

##### Objetivos

1. [Documentacion](https://spark.apache.org/docs/latest/sql-ref-syntax.html) sobre Spark SQL
2. Crear la Base de Datos "demo"
3. Acceder al "**Catalog**" en la "interfaz de Usuario"
4. Comando "**SHOW**"
5. Comando "**DESCRIBE(DESC)**"
6. Mostrar la Base de Dato Actual

In [0]:
-- Podemos usar DATABASE | SCHEMA
CREATE SCHEMA IF NOT EXISTS demo;

In [0]:
-- Mostrar las Base de datos |

SHOW DATABASES;

In [0]:
-- Podemos describir una base de datos

DESCRIBE DATABASE demo;

In [0]:
-- Podemos usar la palabra reservada EXTENDED, para ver mas informacion
DESCRIBE DATABASE  EXTENDED demo;

In [0]:
-- Para saber en que DB estamos posicionados se utiliza la expresion CURRENT_DATABASE
SELECT current_database();

In [0]:
--- Consultar las tablas de una BD
SHOW TABLES IN demo;

In [0]:
-- Cambiar de una base de datos a otra, usamos la palabra reservada  USE
USE demo;

In [0]:
SHOW TABLES IN default

#### Tablas Administradas (Managed Tables)

##### Objetivos

1. Crear una "**Tabla Administrada(Managed Table)**" con Python
2. Crear una "**Tabla Administrada(Managed Table)**" con SQL
3. Efecto de eliminar una Tabla Administrada
4. Describir(Describe) la Tabla

In [0]:
%run "../includes/configuration"

In [0]:
%python
results_movie_genre_language = spark.read.parquet(f"{gold_folder_path}/results_movie_genre_language")

In [0]:
%python
# Crear  una tabla administrada con python
# demo.results_movie_genre_language_python, al indicar demo, estamos indicando que se cree la tabla 
# results_movie_genre_language_python en la base de datos demo
results_movie_genre_language.write.format("parquet").saveAsTable("demo.results_movie_genre_language_python")

In [0]:
USE demo;
SHOW TABLES;

In [0]:
DESCRIBE EXTENDED results_movie_genre_language_python;

In [0]:
SELECT * FROM results_movie_genre_language_python WHERE genre_name = 'Adventure';

In [0]:
-- Tambien podemos crear una tabla administrada a traves de una consulta (CTAS)
-- Se crear una tabla administrada a partir de SQL
CREATE TABLE demo.results_movie_genre_language_sql 
AS
SELECT * FROM results_movie_genre_language_python WHERE genre_name = 'Adventure';

In [0]:
SELECT * FROM results_movie_genre_language_sql;

In [0]:
SELECT CURRENT_DATABASE()

In [0]:
DESCRIBE EXTENDED results_movie_genre_language_sql;

##### Si se elimna una tabla administrada no solamente se van a eliminar los meta datos, si no tambien se van a eliminar los datos

In [0]:
DROP TABLE IF EXISTS results_movie_genre_language_sql;

In [0]:
SHOW TABLES IN demo;

#### Tablas Externas (External Tables)

##### Objetivos

1. Crear una **"Tabla Externa(External Table)"** con Python
2. Crear una **"Tabla Externa(External Table)"** con SQL
3. Efecto de eliminacion de unta **"Tabla Externa(External Table)"**
4. Describir(Describe) la Tabla

In [0]:
%python
# Crear una tabla externa usando Python, lo que hacemos es proporcionarle una ruta donde se guardará la tabla 
# en .option(ruta...)
results_movie_genre_language.write.format("parquet").option("path",f"{gold_folder_path}/results_movie_genre_language_py").saveAsTable("demo.results_movie_genre_language_py")

In [0]:
DESCRIBE EXTENDED results_movie_genre_language_py;

In [0]:
SELECT * FROM demo.results_movie_genre_language_py

In [0]:
-- Generar una tabla externa usando sql
-- Nota: Para que sea una tabla externa tenemos que
-- proporcionar la ruta => LOCATION
CREATE TABLE demo.results_movie_genre_language_sql(
  title STRING,
  duration_time INT,
  release_date DATE,
  vote_average FLOAT,
  language_name STRING,
  genre_name STRING,
  created_date TIMESTAMP
)
USING PARQUET
LOCATION "/mnt/sacjccmoviehistory/gold/results_movie_genre_language_ext_sql";

In [0]:
SHOW TABLES IN demo;

In [0]:
-- La tabla externa "results_movie_genre_language_sql" estara visible en la carpeta gold, hasta que se le inserten datos
INSERT INTO demo.results_movie_genre_language_sql
SELECT * FROM demo.results_movie_genre_language_py
        WHERE genre_name = 'Adventure';

In [0]:
SELECT COUNT(1) FROM demo.results_movie_genre_language_sql;

In [0]:
SHOW TABLES IN demo

In [0]:
-- Esta es una tabla externa, por lo tanto los datos no van a eliminarse de la ubicacion externa, solo se eliminan de los metadatos
-- Como esto es una tabla externa lo que sucede es que Databricks no tiene acceso a estos Datos, unicamente tiene acceso a los metadatos como la estructura de la tabla, los tipos de datos, las columnas, pero no tiene acceso a los datos
DROP TABLE demo.results_movie_genre_language_sql;

### Vistas (Views)

#### Objetivos

1. Crear Vista Temporal
2. Crear vista Temporal Global
3. Crear Vista Permanente 

In [0]:
SELECT current_schema();  

In [0]:
-- Viata Temporal Local
CREATE OR REPLACE TEMP VIEW v_results_movies_genres_language 
AS
SELECT * 
FROM 
demo.results_movie_genre_language_py
WHERE genre_name = 'Adventure';

In [0]:
SELECT * FROM v_results_movies_genres_language;

In [0]:
-- Vista Temporal Global, esta vista global puede ser accedida desde otra Notebook
CREATE OR REPLACE GLOBAL TEMP VIEW gv_results_movies_genres_language 
AS
SELECT * 
FROM 
demo.results_movie_genre_language_py
WHERE genre_name = 'Drama';

In [0]:
SHOW TABLES IN GLOBAL_TEMP;

In [0]:
SELECT * FROM global_temp.gv_results_movies_genres_language

In [0]:
-- Crear vista permanente
CREATE OR REPLACE VIEW pv_results_movies_genres_language 
AS
SELECT * 
FROM 
demo.results_movie_genre_language_py
WHERE genre_name = 'Comedy';

In [0]:
-- La vista "pv_results_movies_genres_language", por ser permanente si se creo dentro de a base de datos demo
SHOW TABLES

In [0]:
SELECT * FROM pv_results_movies_genres_language